In [1]:
import cv2
import numpy as np
import os

import mediapipe as mp

from keras_facenet import FaceNet
from scipy.spatial.distance import cosine



2025-07-11 08:29:39.109378: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

# Load embeddings
data = np.load("faces_centroids_4classes.npz", allow_pickle=True)
X_known, Y_known = data['arr_0'], data['arr_1']

In [3]:
import time

In [4]:

embedder = FaceNet()
mp_face = mp.solutions.face_detection
detector = mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.5)

I0000 00:00:1752236990.258899 7871113 gl_context.cc:357] GL version: 2.1 (2.1 ATI-7.0.3), renderer: AMD Radeon Pro 560X OpenGL Engine


In [14]:
class Person:
    def __init__(self, id, center, name = "Unknown", confidence = 0.0, delete = "False"):
        self.id = id
        self.center = center
        self.name = name
        self.confidence = confidence
        self.delete = delete
        self.smoothed_center = center  # ← initialize smoothed center
        self.alpha = 0.4               # ← smoothing factor (adjust as needed)
    def add_name(self, name):
        self.name = name

    def add_confidence(self, confidence):
        self.confidence = confidence

    def add_center(self, new_center):
        old_cx, old_cy = self.smoothed_center
        new_cx, new_cy = new_center
        smoothed_cx = self.alpha * new_cx + (1 - self.alpha) * old_cx
        smoothed_cy = self.alpha * new_cy + (1 - self.alpha) * old_cy
        self.smoothed_center = (int(smoothed_cx), int(smoothed_cy))
        self.center = new_center  # optionally keep the raw center too
        
    def update_delete(self, delete):
        self.delete = delete
        
    def get_id(self):
        return self.id
    
    def get_center(self):
        return self.smoothed_center
    
    def get_name(self):
        return self.name

    def get_confidence(self):
        return self.confidence

    def get_delete(self):
        return self.delete

#maybe have a check distance function

In [17]:
from utils import streaming_utils_Tut
streaming_utils_Tut.getStream(0)

cap = streaming_utils_Tut.getStream()

In [18]:
#change to objects
start_id_forunknowns = len(Y_known)
next_id = start_id_forunknowns
max_id = start_id_forunknowns + 10
recognize_count = 0

# sets stire the ids and the filter we would do the same thing, go through the object's ids and delete if the id is not in the set
# lets make max list size len(Y_known + 15)
# lets have a master list with a defined size to ensure no duplicates
TrackedPeople_masterlist = [None] * (max_id + 5)
TrackedPeople = []     # List of dicts tracking face info, lets say for in frame
# [{'id': 0, 'center': (x, y), 'label': "Jordan", 'confidence': 0.92}]

# Distance threshold (in pixels) to consider same face
DIST_THRESH = 80
frame_count = 0
# finds the box for the face and finds the center of the face which is used to
# figure out if we are looking at the same person frame by frame without rerecognizing the person
def get_center(box, img_shape):
    h, w = img_shape[:2]
    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (x1, y1, x2, y2), (cx, cy)
# maybe have if runtime is above 100 ms, make new faces unknown
# check match will for each already known person, distance is calculated and compared to our threshold. 
def check_match(box, center, matched_id, img_rgb):
    global next_id
    x1, y1, x2, y2 = box
    cx, cy = center
    label = ""
    confidence = 0.0

    for person in TrackedPeople:
        #print(frame_count, person.get_name(), "1")
        old_cx, old_cy = person.get_center()
        dist = np.linalg.norm([cx - old_cx, cy - old_cy])
        #if dist == 0: # if distance is 0 then its a stillframe duplicate
            #person.update_delete(True)
        label = "Detecting..."
        confidence = 0.0
        if dist < DIST_THRESH: #returning true helps keep frame rate high by recognitizing up to 1 new person per frame
            matched_ids.add(person.get_id()) #add same id
            person.add_center((cx, cy)) #add new center
            if frame_count % 15 == 0: # re recognize face every 15 frames
                # rerecognize face to check for mislabeling
                face_img = img_rgb[y1:y2, x1:x2]
                label, confidence, match_index = recognize(face_img)
                if confidence > person.get_confidence(): # if confidence is greater than current, update
                    person.add_name(label)
                    person.add_confidence(confidence)
            return

    # if its not within distance it must be a different person, adding a checck would take the whole
    # point of the code above away, lets figure out how be more sure than if distance is within range
    # for a given person without increasing our runtime too much
    face_img = img_rgb[y1:y2, x1:x2]
    label, confidence, match_index = recognize(face_img) 
    if match_index == -1:
        match_index = next_id
    person = Person(match_index, (cx, cy), label, confidence, False)
    if TrackedPeople_masterlist[person.get_id()] == None:
        TrackedPeople.append(person)
        TrackedPeople_masterlist[person.get_id()] = person
    matched_ids.add(match_index)
     
    return


def recognize(face_img):
    if face_img.shape[0] > 0 and face_img.shape[1] > 0:
        face_img = img_rgb[y1:y2, x1:x2]
        face_img = cv2.resize(face_img, (160, 160))
        embedding = embedder.embeddings(np.expand_dims(face_img, axis=0))[0]
        distances = [cosine(embedding, emb) for emb in X_known]
        min_dist = min(distances)
        confidence = 1 - min_dist
        if(confidence > 0.6):
            match_index = distances.index(min_dist)
            label = Y_known[match_index]
            return label, confidence, match_index
    if next_id > max_id: #may need to check later for edge case of many unknowns
        next_id = start_id_forunknowns
    next_id += 1
    return "Unknown", 0.0, next_id - 1


# --- Main Loop ---
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.6) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Detect
        results = face_detector.process(img_rgb)

        matched_ids = set()
        if results.detections:
            print(len(results.detections), len(TrackedPeople))
            for detection in results.detections:
                box, center = get_center(detection.location_data.relative_bounding_box, img_rgb.shape)
                x1, y1, x2, y2 = box
                cx, cy = center

                
                check_match(box, center, matched_ids, img_rgb) 
                # Maybe if all faces are accounted for and this unknown still exists then add
                # if not all faces are accounted for then recognize face, if it matches, update it
                # if it doesn't match then we can add a new person
                

        # Remove people not seen this frame (e.g., left frame) 
        # TrackedPeople = [p for p in TrackedPeople if p.get_id() in matched_ids]
        new_tracked_people = []

        for person in TrackedPeople:
            if person.get_id() in matched_ids:
                if person.get_delete() == False:
                    new_tracked_people.append(person)
                else:
                    TrackedPeople_masterlist[person.get_id()] = None
            else:
                TrackedPeople_masterlist[person.get_id()] = None
        TrackedPeople = new_tracked_people
        
        print("recog", recognize_count, frame_count)
        # Draw boxes and labels
        counter = 0
        for person in TrackedPeople:
            counter += 1
            print(counter)
            cx, cy = person.get_center()
            x1, y1 = cx - 100, cy - 100
            x2, y2 = cx + 100, cy + 100
            label = person.get_name()
            confidence = person.get_confidence()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f}) {person.get_id()}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(
            frame,                       # The image to draw on
            f"{label} {person.get_id()}",            # The text to display
            (10, 30 * counter),                    # (x, y) position (top-left corner)
            cv2.FONT_HERSHEY_SIMPLEX,   # Font
            0.8,                           # Font scale (size)
            (0, 255, 0),                 # Color in BGR (green here)
            2,                           # Thickness
            cv2.LINE_AA                 # Line type for better quality
            )


        
        
        cv2.imshow("Real-Time Face Recognition", frame)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


1 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


I0000 00:00:1752240175.028582 7871113 gl_context.cc:357] GL version: 2.1 (2.1 ATI-7.0.3), renderer: AMD Radeon Pro 560X OpenGL Engine
W0000 00:00:1752240175.060663 7955000 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


recog 0 1
1
1 1
recog 0 2
1
1 1
recog 0 3
1
1 1
recog 0 4
1
1 1
recog 0 5
1
1 1
recog 0 6
1
1 1
recog 0 7
1
1 1
recog 0 8
1
1 1
recog 0 9
1
1 1
recog 0 10
1
1 1
recog 0 11
1
1 1
recog 0 12
1
1 1
recog 0 13
1
1 1
recog 0 14
1
1 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
recog 0 15
1
1 1
recog 0 16
1
1 1
recog 0 17
1
1 1
recog 0 18
1
1 1
recog 0 19
1
1 1
recog 0 20
1
1 1
recog 0 21
1
1 1
recog 0 22
1
1 1
recog 0 23
1
1 1
recog 0 24
1
1 1
recog 0 25
1
1 1
recog 0 26
1
1 1
recog 0 27
1
1 1
recog 0 28
1
1 1
recog 0 29
1
1 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
recog 0 30
1
1 1
recog 0 31
1
1 1
recog 0 32
1
1 1
recog 0 33
1
1 1
recog 0 34
1
1 1
recog 0 35
1
1 1
recog 0 36
1
1 1
recog 0 37
1
1 1
recog 0 38
1
1 1
recog 0 39
1
1 1
recog 0 40
1
1 1
recog 0 41
1
1 1
recog 0 42
1
1 1
recog 0 43
1
1 1
recog 0 44
1
1 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
recog 0 45
1
1 1
recog 0 46
1
1 1
recog 0 47
1
1 1
recog 0 48
1
1 1
recog 0 49
1
1 1
recog 0 50
1
1 1
recog 0 51
1
1 1
recog 0 52
1
1 1
recog 0 53


In [15]:
label = Y_known
print(len(label))
for names in Y_known:
    print(names)

10
Tyler
Kendall
robert_downey
Zach
John
sardor_abdirayimov
Pahal
jenna_ortega
Jordan
taylor_swift


In [ ]:
#change to objects
# sets stire the ids and the filter we would do the same thing, go through the object's ids and delete if the id is not in the set

TrackedPeople = []     # List of dicts tracking face info
# list of dicts: [{'id': 0, 'center': (x, y), 'label': "Jordan", 'confidence': 0.92}]
next_id = 0
recognize_count = 0
# Distance threshold (in pixels) to consider same face
DIST_THRESH = 60
frame_count = 0
# finds the box for the face and finds the center of the face which is used to
# figure out if we are looking at the same person frame by frame without rerecognizing the person
def get_center(box, img_shape):
    h, w = img_shape[:2]
    x1 = int(box.xmin * w)
    y1 = int(box.ymin * h)
    x2 = int((box.xmin + box.width) * w)
    y2 = int((box.ymin + box.height) * h)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    return (x1, y1, x2, y2), (cx, cy)
# maybe have if runtime is above 100 ms, make new faces unknown
# check match will for each already known person, distance is calculated and compared to our threshold. 
def check_match(box, center, matched_id, img_rgb):
    global next_id # used for creating an id for unknown people
    for each person in tracked people
    we get out distance from the last frame and see if its less than our distance threshold
    if it is then we update their data with our new distance and add it to match ids
    if frame count is a multiple of 15, we recognize their face as a way to ensure mislabels are fixed
    for person in TrackedPeople:
        #print(frame_count, person.get_name(), "1")
        old_cx, old_cy = person.get_center()
        dist = np.linalg.norm([cx - old_cx, cy - old_cy])
        label = "Detecting..."
        confidence = 0.0
        # edge case to check is what if multiple people are really close: maybe edge cases should be handled in seperate functions
        if dist < DIST_THRESH: # if distance is below distance threshold it must be the same person so we c
            matched_ids.add(person.get_id()) #add same id
            person.add_center((cx, cy)) #add new center
            if frame_count % 15 != 0:
                return True

            # What if new label is unknown
            face_img = img_rgb[y1:y2, x1:x2]
            label, confidence, match_index = recognize(face_img)
            person.add_name(label)
            person.add_confidence(confidence)
            

        
    # maybe issue with multiple faces popping up when I move too fast can be fixed with looking at num detection
    # if num, detections == 1 then maybe 
    face_img = img_rgb[y1:y2, x1:x2]
    label, confidence, match_index = recognize(face_img) 
    if match_index == -1:
        match_index = next_id
    person = Person(match_index, (cx, cy), label, confidence)
    TrackedPeople.append(person)
    matched_ids.add(match_index)
    next_id += 1
            
            
    return False


# def recognize(face_img):
#     face_img comes form check_match
#     uses face_img to compare to all embeddings to get the face with the closest match
#     ie distance using our cosine similarity
#     returns name, confidence, index



# --- Main Loop ---
with mp_face.FaceDetection(model_selection=1, min_detection_confidence=0.6) as face_detector:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # converts image to right format

        # Detect
        results = face_detector.process(img_rgb) # our actual results from the image

        matched_ids = set() # used to clean up our trackedPoeple list
        if results.detections: # if there are detections
            for detection in results.detections: # for each detection
                box, center = get_center(detection.location_data.relative_bounding_box, img_rgb.shape)
                x1, y1, x2, y2 = box
                cx, cy = center

                # each detection is passed to match and will update person if match is true
                # if match is false it will create a new person and add it to the list
                Match = check_match(box, center, matched_ids, img_rgb)
                

        # Remove people not seen this frame (e.g., left frame) 
        TrackedPeople = [p for p in TrackedPeople if p.get_id() in matched_ids]
        print("recog", recognize_count, frame_count)
        # Draw boxes and labels
        counter = 0
        for person in TrackedPeople:
            counter += 1
            print(counter)
            cx, cy = person.get_center()
            x1, y1 = cx - 100, cy - 100
            x2, y2 = cx + 100, cy + 100
            label = person.get_name()
            confidence = person.get_confidence()
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} ({confidence:.2f}) {person.get_id()}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(
            frame,                       # The image to draw on
            f"{label} {person.get_id()}",            # The text to display
            (10, 30 * counter),                    # (x, y) position (top-left corner)
            cv2.FONT_HERSHEY_SIMPLEX,   # Font
            0.8,                           # Font scale (size)
            (0, 255, 0),                 # Color in BGR (green here)
            2,                           # Thickness
            cv2.LINE_AA                 # Line type for better quality
            )


        
        
        cv2.imshow("Real-Time Face Recognition", frame)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()


In [ ]:
# ret : if frame is read secessfully 

# Facenet time is mid 60s ms
# mediaPipes face recognition is 7-11 ms